## Preparing & loading

In [1]:
import copy
from functools import partial
import json
import os
from pathlib import Path
from typing import Callable, Dict, List, Optional, Tuple, Union

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
from torch.nn.utils import clip_grad_norm_
from torch.nn.utils.rnn import pad_sequence
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, Dataset
import torchaudio   
from torchaudio.functional import edit_distance
from torchaudio.models.decoder import ctc_decoder
from tqdm import tqdm


In [2]:
AUDIO_FILES_DIR = Path('/mnt/c/Study/Python/Morse/morse_dataset')
CACHE_DIR = Path("/mnt/c/Study/Python/Morse/mel_cache")
SEED = 42
np.random.seed(SEED)

train_df = pd.read_csv('/mnt/c/Study/Python/Morse/train.csv')
val_df = pd.read_csv('/mnt/c/Study/Python/Morse/val.csv')
test_df = pd.read_csv('/mnt/c/Study/Python/Morse/test.csv')

Получение словаря

In [3]:
def get_vocab(words: pd.Series, blank: str = "<blk>") -> dict[int, str]:
    """
    Строит словарь всех уникальных символов.
    """
    vocab = {0: blank, 1: "|"}
    all_chars = set("".join(words.astype(str)))

    for i, char in enumerate(sorted(all_chars), start=2):
        vocab[i] = char

    print(f"Vocab is ready, size = {len(vocab)}")
    return vocab

In [4]:
full_vocab = get_vocab(train_df['message'])

Vocab is ready, size = 46


Tokenizer для перевода символов в индексы и наоборот

In [5]:
class Tokenizer:
    def __init__(self, vocab: Dict[int, str]):
        self.index_char = vocab
        self.char_index = {char: index for index, char in self.index_char.items()}

    def __call__(
        self, chars: Union[str, List[str]]
    ) -> Union[List[int], List[List[int]]]:
        """
        Преобразует строку или список строк в список индексов.
        """
        if isinstance(chars, str):
            return [self.char_index.get(char, -1) for char in chars]
        elif isinstance(chars, list):
            return [self.__call__(t) for t in chars]
        else:
            raise ValueError("Expected list or str")

    def decode(
        self, indexs: Union[List[int], List[List[int]]]
    ) -> Union[str, List[str]]:
        """
        Преобразует список индексов или список списков индексов обратно в строку(и).
        """
        if isinstance(indexs, list):
            if isinstance(indexs[0], list):
                return [self.decode(i) for i in indexs]
            else:
                return "".join([self.index_char.get(idx, "") for idx in indexs])
        else:
            raise ValueError("Expected list or str")

Готовая функция для перевода выходов модели в конечный набор символов

In [6]:
def decoding_to_tokens(
    decoder: Callable,
    model_output: torch.Tensor,
    tokenizer: Callable,
) -> List[str]:
    """
    Применяет декодер(beam_search) и tokenizer к выходу модели и возвращает расшифрованные строки.
    """

    log_probs = nn.functional.log_softmax(model_output, dim=-1)
    results = decoder(log_probs.contiguous())  # log_probs: [B, T, C]

    decoded_sequences = []
    for batch_result in results:
        top_hypo = batch_result[0]
        tokens = top_hypo.tokens.tolist()
        decoded_sequence = tokenizer.decode(tokens)
        decoded_sequence = decoded_sequence.strip("|")
        decoded_sequences.append(decoded_sequence)

    return decoded_sequences

Адаптация nn.CTCLoss под нашу задачу

In [7]:
def loss_ctc(
    model_output: torch.Tensor,  # Тензор [Batch, Time, Classes] - ожидается на device
    targets: torch.Tensor,  # Тензор [sum(target_lengths)] - ожидается на CPU
    target_lengths: torch.Tensor,  # Тензор [B] - ожидается на CPU
    blank_id: int = 0,
) -> torch.Tensor:
    """
    Вычисляет CTC-лосс (Connectionist Temporal Classification) между выходом модели и целевыми метками.

    """
    log_output = F.log_softmax(model_output, dim=-1)
    log_output = log_output.transpose(0, 1)

    output_time_dim = log_output.shape[0]  # T'
    batch_size = log_output.shape[1]  # B

    output_lengths = torch.full(
        size=(batch_size,), fill_value=output_time_dim, dtype=torch.long, device="cpu"
    )
    targets_cpu = targets.cpu()
    target_lengths_cpu = target_lengths.cpu()

    loss = nn.CTCLoss(blank=blank_id, reduction="mean", zero_infinity=True)

    batch_loss = loss( 
        log_output.float(), targets_cpu, output_lengths, target_lengths_cpu
    )
    return batch_loss

Класс Dataset, принимающий имена файлов, transform для предобработки этих файлов и tokenizer для подготовки target

In [8]:
class MorseDataset(Dataset):
    """
    Класс для загрузки и обработки датасета

    Атрибуты:
        X_filenames (pd.Series): Список путей к аудио файлам.
        y_texts (Optional[pd.Series]): Список целевых меток (строки).
        transform (Optional[Callable]): Функция для преобразования аудио файлов в признаки.
        tokenizer (Optional[Callable[[str], list]]): Tokenizer для преобразования текста в индексы.
    """

    def __init__(
        self,
        X_filenames: pd.Series,
        y_texts: Optional[pd.Series] = None,
        transform: Optional[Callable] = None,
        tokenizer: Optional[Callable[[str], list]] = None,
    ):
        self.X_filenames = X_filenames.reset_index(drop=True)
        self.y_texts = y_texts.reset_index(drop=True) if y_texts is not None else None
        self.transform = transform
        self.tokenizer = tokenizer

    def __len__(self) -> int:
        return len(self.X_filenames)

    def __getitem__(self, idx) -> Dict[str, torch.Tensor]:
        filename = self.X_filenames[idx]
        mel_features = self.transform(filename)  # [C, T] ([n_mels, time])

        item = {"input": torch.tensor(mel_features, dtype=torch.float)}

        if self.y_texts is not None:
            text = self.y_texts[idx]
            if self.tokenizer:
                target = self.tokenizer(text)
            else:
                raise ValueError("Tokenizer needs for target encoding")
            item["target_text"] = text
            item["target"] = torch.tensor(target, dtype=torch.long)
            item["target_length"] = len(target)

        return item

Функции для создания DataLoader с кастомным collate_fn для создания удобного batch при работе с CTCLoss

In [9]:
def dataloader_collate(batch: List[Dict[str, torch.Tensor]]) -> Dict[str, torch.Tensor]:
    """
    Функция для объединения элементов батча, с добавлением паддинга для последовательностей.
    """
    inputs = [item["input"].T for item in batch]  # [T, C]

    padded_inputs = pad_sequence(inputs, batch_first=True)  # [B, max_T, C]
    padded_inputs = padded_inputs.transpose(1, 2)  # [B, C, T]

    collated = {
        "input": padded_inputs,
    }

    if "target" in batch[0]:
        targets = torch.cat([item["target"] for item in batch], dim=0)
        target_lengths = torch.tensor(
            [item["target_length"] for item in batch], dtype=torch.long
        )
        collated["target_text"] = [item["target_text"] for item in batch]
        collated["target"] = targets
        collated["target_length"] = target_lengths

    return collated


def data_loader(
    dataset: torch.utils.data.Dataset,
    batch_size: int,
    shuffle: bool,
    num_workers: int = 0,
    drop_last: bool = True,
) -> DataLoader:
    """
    Функция для создания DataLoader с заданными параметрами.
    """
    return DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        collate_fn=dataloader_collate,
        num_workers=num_workers,
        drop_last=drop_last,
        pin_memory=True,
    )

Основная функция для обучения модели и валидации результатов. <br>
Для обучения используется CTC-loss. Валидация проводится с использованием beam-search декодирования, <br>
после чего рассчитывается расстояние Левенштейна между предсказанным и истинным текстом.<br>

In [10]:
def train_model(
    model: nn.Module,
    train_loader: DataLoader,
    optimizer: torch.optim.Optimizer,
    ctc_loss: Callable,
    epochs: int,
    metric: Callable,
    decoder: Callable,
    tokenizer: Callable,
    val_loader: Union[DataLoader, None] = None,
    scheduler: Union[Callable, None] = None,
    save_path: str = "best_model.pt",
) -> Tuple[nn.Module, Dict[str, List]]:
    """
    Args:
        model: Нейронная сеть для обучения.
        train_loader: DataLoader для тренировочных данных.
        optimizer: Оптимизатор для обновления параметров модели.
        ctc_loss: Функция CTC-лосса.
        epochs: Количество эпох обучения.
        metric: Функция для оценки качества предсказаний (например, edit_distance).
        decoder: Функция декодирования CTC-выхода в токены.
        tokenizer: Функция токенизации текста.
        val_loader: DataLoader для валидационных данных (опционально).
        scheduler: Планировщик скорости обучения (опционально).
        save_path: Путь для сохранения лучшей модели (по умолчанию "best_model.pt").

    Returns:
        Tuple containing:
        - Trained model (nn.Module)
        - Dictionary with metrics ("train_loss", "val_loss", "val_metric")
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    model.to(device, non_blocking=True)
    metrics = {"train_loss": [], "val_loss": [], "val_metric": []}
    
    best_val_metric = float("inf")  
    best_model_state = None

    for epoch in range(epochs):
        print(f"\n Epoch {epoch + 1}/{epochs}")
        epoch_loss = []
        model.train()
        with tqdm(
            train_loader, desc="Training", total=len(train_loader), dynamic_ncols=True
        ) as pbar:
            for batch in pbar:
                inputs = batch["input"].to(device, non_blocking=True)  # (B, C, T)
                targets = batch["target"]  # (sum_target_len,)
                target_lengths = batch["target_length"]  # (B,)
                optimizer.zero_grad()

                output = model(inputs)
                loss = ctc_loss(output, targets, target_lengths)

                loss.backward()
                clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()
                epoch_loss.append(loss.item())

        avg_epoch_loss = np.mean(epoch_loss)
        metrics["train_loss"].append(avg_epoch_loss)

        print(f"Epoch {epoch + 1} with loss = {avg_epoch_loss:.4f}")
        if scheduler is not None:
            scheduler.step()
            print(f"Learning_rate = {scheduler.get_last_lr()}")

        if val_loader is not None:
            model.eval()
            epoch_val_metrics = []
            epoch_val_losses = []

            with torch.no_grad():
                for batch in tqdm(
                    val_loader, desc=f"Validation Epoch {epoch + 1}", leave=False
                ):
                    inputs = batch["input"].to(device, non_blocking=True)  # (B, C, T)
                    target_text = batch["target_text"]
                    targets = batch["target"]  # (sum_target_len,)
                    target_lengths = batch["target_length"]  # (B,)
                    predictions = model(inputs)
                    
                    val_loss = ctc_loss(predictions, targets, target_lengths)
                    epoch_val_losses.append(val_loss.item())

                    emissions = predictions.detach().cpu().float()

                    tokens_output = decoding_to_tokens(decoder, emissions, tokenizer)
                    batch_val_metrics = [
                        metric(pred, gt) for pred, gt in zip(tokens_output, target_text)
                    ]
                    epoch_val_metrics.append(np.mean(batch_val_metrics))

            avg_epoch_val_metrics = np.mean(epoch_val_metrics)
            avg_epoch_val_loss = np.mean(epoch_val_losses)
            metrics["val_loss"].append(avg_epoch_val_loss)
            metrics["val_metric"].append(avg_epoch_val_metrics)
            print(f"Val Loss={avg_epoch_val_loss:.4f}, Val Metric={avg_epoch_val_metrics:.4f}")
            
            if avg_epoch_val_metrics < best_val_metric:
                best_val_metric = avg_epoch_val_metrics
                best_model_state = model.state_dict()
                torch.save(best_model_state, save_path)
                print(f"Saved best model with Val Metric={best_val_metric:.4f} at {save_path}")

        else:
            metrics["val_loss"].append(None)
            metrics["val_metric"].append(None)

    return model, metrics

Блоки для создания модели - сверточный блок и LSTM-блок

In [11]:
class ConvBlock(nn.Module):
    """
    Свёрточный блок с нормализацией и активацией ReLU.

    Включает свёртку (Conv1d), слой нормализации (LayerNorm) и активацию ReLU.
    """

    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        kernel_size: int,
        padding: int,
        stride: int = 1,
        groups: int = 1,
    ):
        super().__init__()
        self.conv = nn.Conv1d(
            in_channels,
            out_channels,
            kernel_size,
            padding=padding,
            stride=stride,
            groups=groups,
        )
        self.norm = nn.LayerNorm(out_channels)
        self.relu = nn.ReLU()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.conv(x)
        x = self.norm(x.transpose(1, 2)).transpose(1, 2)
        return self.relu(x)


class LstmBlock(nn.Module):
    """
    Блок LSTM с нормализацией и dropout.

    Включает двустороннюю LSTM (2 слоя), слой нормализации (LayerNorm) и слой dropout.
    """

    def __init__(self, input_size: int, hidden_size: int, dropout: float = 0.1):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=2,
            bidirectional=True,
            batch_first=True,
        )
        self.norm = nn.LayerNorm(hidden_size * 2)
        self.drop = nn.Dropout(dropout)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x, _ = self.lstm(x)
        x = self.drop(self.norm(x))
        return x

Инициализация начальных весов модели в зависимости от блока.

In [12]:
def init_weights(module: nn.Module) -> None:
    """
    Инициализация весов для различных типов слоёв.
    """
    if isinstance(module, nn.Conv1d):
        init.kaiming_normal_(module.weight, nonlinearity="relu")
        if module.bias is not None:
            init.constant_(module.bias, 0)

    elif isinstance(module, nn.Linear):
        init.xavier_uniform_(module.weight)
        if module.bias is not None:
            init.constant_(module.bias, 0)

    elif isinstance(module, nn.LSTM):
        for name, param in module.named_parameters():
            if "weight_ih" in name:
                init.xavier_uniform_(param)
            elif "weight_hh" in name:
                init.orthogonal_(param)
            elif "bias" in name:
                init.constant_(param, 0)

    elif isinstance(module, nn.LayerNorm):
        if module.elementwise_affine:
            init.constant_(module.weight, 1.0)
            init.constant_(module.bias, 0.0)

Отрисовка словаря с метриками, полученного после функции train_model()

In [13]:
def plot_metrics(
    metrics_dict: dict,
    title: str = "Model Metrics",
    xlabel: str = "Epochs",
    ylabel: str = "Metric Value",
) -> None:
    """
    Функция для отображения метрик в виде графиков.
    """
    plt.figure(figsize=(10, 6))
    epochs = range(1, len(list(metrics_dict.values())[0]) + 1)

    for metric_name, metric_values in metrics_dict.items():
        if metric_values:
            plt.plot(epochs, metric_values, label=metric_name)

    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.legend(loc="best")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [14]:
def path_to_blackmel_cached(
    file_name: str,
    audio_files_dir: Path,
    cache_dir: Path,
    overwrite: bool = False,
) -> np.ndarray:
    cache_path = cache_dir / (file_name.replace(".wav", ".npy"))

    if cache_path.exists() and not overwrite:
        return np.load(cache_path)

    waveform, sr = torchaudio.load(audio_files_dir / file_name)

    mel = torchaudio.transforms.MelSpectrogram(
        sample_rate=sr, n_fft=512, hop_length=128, n_mels=64
    )(waveform)
    boost_mel = mel**5
    boost_mel = boost_mel / boost_mel.max()
    mel_db = torchaudio.transforms.AmplitudeToDB(top_db=64)(boost_mel)
    mel_db = mel_db.squeeze(0).cpu().numpy()

    peak = np.argmax(mel_db.mean(axis=1))
    start = max(peak - 10, 0)
    end = min(peak + 10, mel_db.shape[0])
    fresh_mel = mel_db[start:end]

    os.makedirs(cache_dir, exist_ok=True)
    np.save(cache_path, fresh_mel)
    return fresh_mel

In [15]:
class MorseDecoder(nn.Module):
    def __init__(
        self,
        input_size: Tuple[int, int],
        kernel_chanel: int,
        conv_out_size: int,
        vocab_size: int,
    ):
        super().__init__()

        self.conv_part = nn.Sequential(
            ConvBlock(
                in_channels=input_size[0],
                out_channels=kernel_chanel,
                kernel_size=5,
                padding=2,
                stride=2,
            ),
            ConvBlock(
                in_channels=kernel_chanel,
                out_channels=conv_out_size,
                kernel_size=4,
                padding=0,
                stride=2,
            ),
            ConvBlock(
                in_channels=conv_out_size,
                out_channels=conv_out_size,
                kernel_size=3,
                padding=0,
            ),
        )

        self.rnn_part = nn.Sequential(
            LstmBlock(input_size=conv_out_size, hidden_size=64, dropout=0.4),
        )

        self.clf = nn.Linear(
            in_features=64 * 2,
            out_features=vocab_size,
        )

    def forward(self, x):
        x = self.conv_part(x)
        x = x.transpose(1, 2)
        x = self.rnn_part(x)
        return self.clf(x)

In [16]:
beam_search_decoder = ctc_decoder(
    lexicon=None,
    tokens=list(full_vocab.values()),
    beam_size=3,
    nbest=1,
    blank_token="<blk>",
)

In [17]:
tokenizer = Tokenizer(vocab=full_vocab)

print(tokenizer.index_char)
print(tokenizer.char_index)

transform = partial(
    path_to_blackmel_cached,
    audio_files_dir=AUDIO_FILES_DIR,
    cache_dir=CACHE_DIR,
)

{0: '<blk>', 1: '|', 2: ' ', 3: '#', 4: '0', 5: '1', 6: '2', 7: '3', 8: '4', 9: '5', 10: '6', 11: '7', 12: '8', 13: '9', 14: 'А', 15: 'Б', 16: 'В', 17: 'Г', 18: 'Д', 19: 'Е', 20: 'Ж', 21: 'З', 22: 'И', 23: 'Й', 24: 'К', 25: 'Л', 26: 'М', 27: 'Н', 28: 'О', 29: 'П', 30: 'Р', 31: 'С', 32: 'Т', 33: 'У', 34: 'Ф', 35: 'Х', 36: 'Ц', 37: 'Ч', 38: 'Ш', 39: 'Щ', 40: 'Ъ', 41: 'Ы', 42: 'Ь', 43: 'Э', 44: 'Ю', 45: 'Я'}
{'<blk>': 0, '|': 1, ' ': 2, '#': 3, '0': 4, '1': 5, '2': 6, '3': 7, '4': 8, '5': 9, '6': 10, '7': 11, '8': 12, '9': 13, 'А': 14, 'Б': 15, 'В': 16, 'Г': 17, 'Д': 18, 'Е': 19, 'Ж': 20, 'З': 21, 'И': 22, 'Й': 23, 'К': 24, 'Л': 25, 'М': 26, 'Н': 27, 'О': 28, 'П': 29, 'Р': 30, 'С': 31, 'Т': 32, 'У': 33, 'Ф': 34, 'Х': 35, 'Ц': 36, 'Ч': 37, 'Ш': 38, 'Щ': 39, 'Ъ': 40, 'Ы': 41, 'Ь': 42, 'Э': 43, 'Ю': 44, 'Я': 45}


In [ ]:
train_dataset = MorseDataset(
    X_filenames=train_df['id'],
    y_texts=train_df['message'],
    transform=transform,
    tokenizer=tokenizer,
)

train_loader = data_loader(
    train_dataset, batch_size=64, shuffle=True, drop_last=True, num_workers=2
)

val_dataset = MorseDataset(
    X_filenames=val_df['id'], y_texts=val_df['message'], transform=transform, tokenizer=tokenizer
)

val_loader = data_loader(
    val_dataset, batch_size=64, shuffle=False, drop_last=True, num_workers=2
)

### Train

In [19]:
cnn_blackmel_model = MorseDecoder(
    input_size=[20, 501],
    kernel_chanel=32,
    conv_out_size=64,
    vocab_size=len(full_vocab),
)
torch.manual_seed(SEED)
cnn_blackmel_model.apply(init_weights)

optimizer = torch.optim.Adam(cnn_blackmel_model.parameters(), lr=0.005)

step_lr = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

cnn_blackmel_model

MorseDecoder(
  (conv_part): Sequential(
    (0): ConvBlock(
      (conv): Conv1d(20, 32, kernel_size=(5,), stride=(2,), padding=(2,))
      (norm): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
      (relu): ReLU()
    )
    (1): ConvBlock(
      (conv): Conv1d(32, 64, kernel_size=(4,), stride=(2,))
      (norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (relu): ReLU()
    )
    (2): ConvBlock(
      (conv): Conv1d(64, 64, kernel_size=(3,), stride=(1,))
      (norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (relu): ReLU()
    )
  )
  (rnn_part): Sequential(
    (0): LstmBlock(
      (lstm): LSTM(64, 64, num_layers=2, batch_first=True, bidirectional=True)
      (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (drop): Dropout(p=0.4, inplace=False)
    )
  )
  (clf): Linear(in_features=128, out_features=46, bias=True)
)

In [21]:
cnn_blackmel_model, cnn_blackmel_metrics = train_model(
    model=cnn_blackmel_model,
    train_loader=train_loader,
    optimizer=optimizer,
    ctc_loss=loss_ctc,
    epochs=40,
    metric=edit_distance,
    decoder=beam_search_decoder,
    tokenizer=tokenizer,
    val_loader=val_loader,
    scheduler=step_lr,
)

Using device: cuda

 Epoch 1/40


Training: 100%|██████████| 421/421 [13:01<00:00,  1.86s/it]


Epoch 1 with loss = 4.0412
Learning_rate = [0.005]


Val Loss=4.0846, Val Metric=8.8991
Saved best model with Val Metric=8.8991 at best_model.pt

 Epoch 2/40


Training: 100%|██████████| 421/421 [01:57<00:00,  3.60it/s]


...

Training: 100%|██████████| 421/421 [01:54<00:00,  3.68it/s]


Epoch 19 with loss = 3.9985
Learning_rate = [0.0025]


Val Loss=4.0142, Val Metric=8.7048

 Epoch 20/40


Training:  82%|████████▏ | 344/421 [01:33<00:21,  3.66it/s]


KeyboardInterrupt: 

In [ ]:
def model_predict(
    model: nn.Module,
    test_paths: pd.Series,
    tokenizer: Tokenizer,
    decoder: Callable,
    batch_size: int = 32,
    device: Optional[torch.device] = None,
) -> pd.DataFrame:
    """
    Parameters:
        model (nn.Module): Обученная модель для предсказаний.
        test_paths (pd.Series): Имена тестовых аудиофайлов.
        tokenizer (Tokenizer): Tokenizer для декодирования индексов.
        decoder(Callable): Декодер для преобразования выходов модели в индексы словаря.
        batch_size (int): Размер батча для DataLoader.
        device (torch.device): Устройство (CPU или CUDA), на котором будет происходить вычисление.

    Returns:
        pd.DataFrame: DataFrame с декодированными предсказаниями.
    """

    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model.to(device)
    model.eval()

    test_dataset = MorseDataset(
        X_filenames=test_paths, transform=transform, tokenizer=tokenizer
    )

    test_loader = data_loader(
        test_dataset, batch_size=batch_size, shuffle=False, drop_last=False
    )

    predictions = []

    with torch.no_grad():
        with tqdm(test_loader, desc="Predicting", total=len(test_loader)) as pbar:
            for batch in pbar:
                inputs = batch["input"].to(device)  # [B, C, T]

                outputs = model(inputs)

                decoded_preds = decoding_to_tokens(decoder, outputs, tokenizer)

                predictions.extend(decoded_preds)

                pbar.set_postfix({"Predictions": len(predictions)})

    return pd.DataFrame({"id": test_paths, "message": predictions})